In [2]:
import pandas as pd

In [3]:
import sys 
sys.path.append("src")
import src.soporte_limpieza as sp

In [5]:
pd.options.display.max_columns = None

# LIMPIEZA DE DATOS

En este notebook se procederá a la limpieza, unificación y preparación para el algoritmo de recomendación de la información extraída de cada una de las fuentes recogidas en el documento 1_Extraccion.ipynb

## 1. Limpieza de Fever

In [35]:
df_fever= pd.read_csv('../data/FEVER.csv')
df_fever_detalle= pd.read_csv('../data/FEVER_DETALLE.csv')

In [36]:
#Extraemos las sku de las url para cruzar los 2 df
df_fever['sku'] = df_fever['url'].apply(sp.Limpieza.extract_sku).astype(int)

#Agrupamos los df
df_fever_all = pd.merge(df_fever, df_fever_detalle, on='sku', how='left')

#Gestionaremos los nulos en la columna descripción reemplazándolos por el nombre de la actividad
df_fever_all['descripcion'].fillna(df_fever_all['nombre'], inplace=True)

#Renombrado de columnas
df_fever_all.rename(columns={'categoria': 'subcategoria'}, inplace=True)

#Eliminación de la columnas que no se utilizarán en este caso
df_fever_all = df_fever_all.drop(['fecha', 'sku'], axis=1)

#Limpieza de la columna de precio
df_fever_all['precio'] = df_fever_all['precio'].apply(sp.Limpieza.columna_precio_fever)

#Limpieza de la columna de rating
df_fever_all['rating'] = df_fever_all['rating'].apply(lambda x: sp.Limpieza.columna_rating(x))

#Renombrado de columnas
df_fever_all.rename(columns={'lugar': 'direccion'}, inplace=True)

#Eliminación de aquellos planes que contienen nulos en el resto de las columnas, puesto que no se pueden reemplazar
df_fever_all = df_fever_all.dropna()

In [37]:
df_fever_all.head(1)

,subcategoria,nombre,url,precio,rating,descripcion,imagen,latitud,longitud,direccion
0,conciertos-festivales,Candlelight Orquesta: lo mejor de Joe Hisaishi,https://feverup.com/m/127269,15.0,0.0,elight son los conciertos a la luz de las vel...,https://applications-media.feverup.com/image/u...,40.420915,-3.7188,Gran Teatro CAIXABANK Príncipe Pío


In [38]:
df_fever_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 884 entries, 0 to 911
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   subcategoria  884 non-null    object 
 1   nombre        884 non-null    object 
 2   url           884 non-null    object 
 3   precio        884 non-null    float64
 4   rating        884 non-null    float64
 5   descripcion   884 non-null    object 
 6   imagen        884 non-null    object 
 7   latitud       884 non-null    float64
 8   longitud      884 non-null    float64
 9   direccion     884 non-null    object 
dtypes: float64(4), object(6)
memory usage: 76.0+ KB


## 2. Limpieza de Groupon

In [39]:
df_groupon= pd.read_csv('../data/GROUPON.csv')
df_groupon_detalle= pd.read_csv('../data/GROUPON_DETALLE.csv')

In [40]:
#Eliminamos la columna precio_2 porque no nos aporta información
df_groupon = df_groupon.drop(['precio_2'], axis=1)

#Agrupamos los df
df_groupon_all = pd.merge(df_groupon, df_groupon_detalle, on='url', how='left')

#Gestionaremos los nulos en la columna descripción reemplazándolos por el nombre de la actividad
df_groupon_all['descripcion'].fillna(df_groupon_all['nombre'], inplace=True)

#Limpieza y gestión de nulos de la columna rating
df_groupon_all['rating'] = df_groupon_all['rating'].apply(lambda x: sp.Limpieza.columna_rating(x))

#Obetenemos la latitud y longitud de las direcciones de las actividades
df_groupon_all[['latitud', 'longitud']] = df_groupon_all['direccion'].apply(lambda x: pd.Series(sp.Limpieza.obtener_lat_long(x)))

#Renombrado de columnas
df_groupon_all.rename(columns={'categoria': 'subcategoria'}, inplace=True)

In [41]:
df_groupon_all.head(1)

,subcategoria,nombre,direccion,url,precio,rating,descripcion,imagen,latitud,longitud
0,juegos-de-escape,Escape room urbano para grupos de 2 a 6 person...,"Calle Manzana, 15, Madrid",https://www.groupon.es/deals/parapark-madrid-2,39.99,4.7,Las opciones incluyen:\nJuego de escape room u...,https://img.grouponcdn.com/deal/2g4EpyyacTyhbh...,0.0,0.0


In [42]:
df_groupon_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98 entries, 0 to 97
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   subcategoria  98 non-null     object 
 1   nombre        98 non-null     object 
 2   direccion     98 non-null     object 
 3   url           98 non-null     object 
 4   precio        98 non-null     float64
 5   rating        98 non-null     float64
 6   descripcion   98 non-null     object 
 7   imagen        98 non-null     object 
 8   latitud       98 non-null     float64
 9   longitud      98 non-null     float64
dtypes: float64(4), object(6)
memory usage: 8.4+ KB


## 3. Limpieza de Tripadvisor

In [43]:
df_tripadvisor= pd.read_json('tutorial/TRIPADVISOR_D.json')
df_tripadvisor_detalle= pd.read_csv('../data/TRIPADVISOR_DETALLE.csv')

In [44]:
#Eliminacion de columnas innecesarias y campos nulos 
df_tripadvisor = df_tripadvisor.drop(['province','price','city', 'price_all','latitude','longitude','meals','specialDiets','locationAll','coordinates','positionlink'], axis=1)
df_tripadvisor = df_tripadvisor.dropna()
df_tripadvisor = df_tripadvisor.reset_index(drop=True)

In [45]:
#Creación la columna subcategoria
df_tripadvisor['subcategoria'] = 'restaurantes'

#Renombrado de columnas
df_tripadvisor.rename(columns={'cuisines': 'descripcion', 'name': 'nombre', 'direction': 'direccion'}, inplace=True)

#Modificación de la columna url para cruzar los 2 df
df_tripadvisor['url'] = 'https://www.tripadvisor.es' + df_tripadvisor['url']

#Agrupamos los df
df_tripadvisor_all = pd.merge(df_tripadvisor, df_tripadvisor_detalle, on='url', how='left')

#Eliminación de los nulos del df en las columnas en las que no hay posibilidad de reemplazo
df_tripadvisor_all = df_tripadvisor_all.dropna()
df_tripadvisor_all = df_tripadvisor_all.reset_index(drop=True)

#Extracción de latitud y longitud de la columna de coordenadas del df
df_tripadvisor_all[['latitud', 'longitud']] = df_tripadvisor_all['coordenadas'].apply(sp.Limpieza.extraer_coordenadas).apply(pd.Series)

#Limpieza de la columna de precios haciendo la media del rango
df_tripadvisor_all['precio'] = df_tripadvisor_all["priceRangeNum"].apply(sp.Limpieza.calcular_media_precio)

#Limpieza de la columna de rating
df_tripadvisor_all['rating'] = df_tripadvisor_all['rating'].apply(sp.Limpieza.extraer_rating_float)

#Eliminar columnas previas a la limpieza
df_tripadvisor_all = df_tripadvisor_all.drop(['coordenadas','priceRangeNum'], axis=1)

In [46]:
df_tripadvisor_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384 entries, 0 to 1383
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   nombre        1384 non-null   object 
 1   url           1384 non-null   object 
 2   direccion     1384 non-null   object 
 3   descripcion   1384 non-null   object 
 4   rating        1384 non-null   float64
 5   subcategoria  1384 non-null   object 
 6   imagen        1384 non-null   object 
 7   latitud       1384 non-null   float64
 8   longitud      1384 non-null   float64
 9   precio        1384 non-null   float64
dtypes: float64(4), object(6)
memory usage: 108.2+ KB


In [47]:
df_tripadvisor_all.head(1)

,nombre,url,direccion,descripcion,rating,subcategoria,imagen,latitud,longitud,precio
0,Los Montes de Galicia,https://www.tripadvisor.es/Restaurant_Review-g...,"Calle Azcona 46 PISO 8, 28028 Madrid España","Española, Internacional, Mediterránea, Saludable",5.0,restaurantes,"data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",40.434765,-3.668304,65.0


## 4. Creación de df_all

In [48]:
#Reestablecer el orden de las columnas para que sea común
df_fever_all = df_fever_all[['nombre', 'subcategoria', 'descripcion', 'url', 'direccion','rating','imagen','latitud','longitud','precio']]
df_groupon_all = df_groupon_all[['nombre', 'subcategoria', 'descripcion', 'url', 'direccion','rating','imagen','latitud','longitud','precio']]
df_tripadvisor_all = df_tripadvisor_all[['nombre', 'subcategoria', 'descripcion', 'url', 'direccion','rating','imagen','latitud','longitud','precio']]

In [49]:
#Concatenado de los df de las diferentes fuentes
df_all = pd.concat([df_fever_all, df_groupon_all, df_tripadvisor_all], ignore_index=True)

#Creación de identificador único por plan a partir de la columna de url
df_all['id'], _ = pd.factorize(df_all['url'], sort=True)

 * Creación de la tabla de categorías- subcategorías 

In [50]:
df_cat_sub = df_all.copy()
df_cat_sub.drop(df_cat_sub.columns.difference(['id', 'subcategoria']), axis=1, inplace=True)

In [51]:
#Importamos el fichero con la relación categoría-subcategoría y añadimos la columna de categoría a df_cat_sub
rel_categoria= pd.read_csv('../data/CATEGORIA_RELACION.csv')
df_cat_sub = pd.merge(df_cat_sub, rel_categoria, on='subcategoria', how='left')
df_cat_sub['subcategoria'] = df_cat_sub['subcategoria'].apply(lambda x: x.replace('-', ' '))

In [52]:
df_cat_sub.to_csv(f'../data/CATEGORIAS.csv', header=True, index=False)

 * Eliminación de duplicados y creación de la tabla principal

In [53]:
df_all= df_all.drop(['subcategoria'], axis=1)
df_principal = df_all.drop_duplicates()

In [54]:
df_principal.to_csv(f'../data/PRINCIPAL.csv', header=True, index=False)

 * Creación de la tabla de entrenamiento

In [55]:
df_entrenamiento = df_principal.copy()

#concatenado de descripcion y nombre
df_entrenamiento['descripcion_conc'] = df_entrenamiento.apply(lambda row: str(row['nombre']) +' '+ str(row['descripcion']), axis=1)

#limpieza de descripcion
df_entrenamiento['descripcion_conc'] = df_entrenamiento['descripcion_conc'].apply(lambda x: sp.Limpieza.limpiar_texto(x))

#Eliminación de columnas
df_entrenamiento.drop(df_entrenamiento.columns.difference(['id', 'descripcion_conc']), axis=1, inplace=True)

In [56]:
df_entrenamiento.to_csv(f'../data/ENTRENAMIENTO.csv', header=True, index=False)